In [1]:
%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=9000)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy
import shutil

#import imp
from scan_gym import envs
#imp.reload(envs)


seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

GPUs:  1


In [2]:
current_path = os.getcwd()
params_file = os.path.join(current_path, 'params.json') 
pm=json.load(open(params_file))
run_label = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
data_log_path = os.path.join(current_path, 'generated_data/') 

In [3]:
data_paths = ['/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/000',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/001',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/003',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/006']

envs = [suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp}) 
        for dp in data_paths]

tf_env = tf_py_environment.TFPyEnvironment( batched_py_environment.BatchedPyEnvironment(envs,multithreading=True))#, isolation=True  )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
tf_env.observation_spec()

(BoundedTensorSpec(shape=(82, 82, 2), dtype=tf.float32, name='observation/tuple_0', minimum=array(0., dtype=float32), maximum=array(255., dtype=float32)),
 BoundedTensorSpec(shape=(5,), dtype=tf.float32, name='observation/tuple_1', minimum=array([-400.,    0.,    0.,  -20.,    0.], dtype=float32), maximum=array([400.,   8.,  20.,  20.,   1.], dtype=float32)))

In [5]:
def image_layers():
    input_img = keras.layers.Input(shape=(82,82,2))
    '''x = keras.layers.Lambda(lambda x: x * 0)(input_img)
    x = keras.layers.Conv2D(filters=1,
                            kernel_size=1,
                            padding="same",
                            strides=(82,82),
                            activation='relu')(x)'''
    
    preprocessing = keras.layers.Lambda(lambda x: x / 255.)(input_img)
    x = keras.layers.Conv2D(filters=32,
                            kernel_size=8,
                            padding="same",
                            strides=(4,4),
                            activation='relu')(preprocessing)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=4,
                            padding="same",
                            strides=(2,2),
                            activation='relu')(x)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=3,
                            padding="same",
                            strides=(1,1),
                            activation='relu')(x)
    x = keras.layers.Flatten()(x)
    model = keras.models.Model(inputs=input_img,outputs=x)
    return model
    
    
    
#scale range 0 to 1
oldmin = tf_env.observation_spec()[0].minimum
oldmax = tf_env.observation_spec()[0].maximum
    
def input_vect_layers():
    input_ = keras.layers.Input(shape=(5,))
    preprocessing = keras.layers.Lambda(lambda x: ((x-oldmin)*(1.- 0.)/(oldmax-oldmin)) + 0. )(input_)
    #x = keras.layers.Dense(4)(preprocessing)
    return keras.models.Model(inputs=input_,outputs=preprocessing)

In [6]:
#network
preprocessing_layers=(image_layers(),input_vect_layers())
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
dense_l = pm['model']['fc_layer_params']
if len(dense_l) == 1:
    fc_layer_params = (dense_l[0],)
else:
    fc_layer_params = dense_l

q_net = QNetwork(
tf_env.observation_spec(),
tf_env.action_spec(),
preprocessing_layers=preprocessing_layers,
preprocessing_combiner=preprocessing_combiner,
fc_layer_params=fc_layer_params)

WARNING - 2021-04-12 18:38:21,821 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-04-12 18:38:21,841 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


In [7]:
#agent
train_step = tf.Variable(0)
#optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
#            epsilon=0.00001, centered=True)
'''lr_decay =  keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=0.005, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.0005)
optimizer = keras.optimizers.Adam(learning_rate=lambda: lr_decay(train_step))'''


optimizer = keras.optimizers.Adam(learning_rate=pm['model']['learning_rate'])

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.01) # final ε

agent = DqnAgent(tf_env.time_step_spec(),
                tf_env.action_spec(),
                q_network=q_net,
                optimizer=optimizer,
                target_update_period=pm['agent']['target_update_period'],
                td_errors_loss_fn= element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                gamma=pm['agent']['gamma'], # discount factor
                train_step_counter=train_step,
                epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()


WARNING - 2021-04-12 18:38:31,652 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-04-12 18:38:31,673 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


In [8]:
#Replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec= agent.collect_data_spec,
    batch_size= tf_env.batch_size,
    max_length=pm['rbuffer']['max_length'])

In [9]:
#observer
#observer is just a function (or a callable object) that takes a trajectory argument,
#add_method() method (bound to the replay_buffer object) can be used as observer
replay_buffer_observer = replay_buffer.add_batch

In [10]:
#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(batch_size=len(envs)),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(batch_size=len(envs)),
tf_metrics.MaxReturnMetric(batch_size=len(envs)),
tf_metrics.MinReturnMetric(batch_size=len(envs)),   
]


'''#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(),
tf_metrics.MaxReturnMetric(),
tf_metrics.MinReturnMetric(),   
]'''

'#observer for training metrics\ntraining_metrics = [\ntf_metrics.NumberOfEpisodes(),\ntf_metrics.AverageEpisodeLengthMetric(),\ntf_metrics.EnvironmentSteps(),\ntf_metrics.AverageReturnMetric(),\ntf_metrics.MaxReturnMetric(),\ntf_metrics.MinReturnMetric(),   \n]'

In [11]:
#custom observer
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [12]:
#Collect Driver
update_period = pm['collect_driver']['num_steps'] # train the model every x steps
collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect x steps for each training iteration

#+ training_metrics,

In [13]:
# random policy driver to start filling the buffer
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)
#, ShowProgress(ns)],
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


WARNING - 2021-04-12 18:38:32,453 - deprecation - From /home/pico/.local/lib/python3.8/site-packages/tf_agents/drivers/dynamic_step_driver.py:196: calling while_loop_v2 (from tensorflow.python.ops.control_flow_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [14]:
#use buffer as tf API dataset ()
dataset = replay_buffer.as_dataset(
        sample_batch_size=pm['rbuffer']['sample_batch_size'],
        num_steps=2,
        num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


WARNING - 2021-04-12 18:38:48,944 - api - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/autograph/operators/control_flow.py:1218: ReplayBuffer.get_next (from tf_agents.replay_buffers.replay_buffer) is deprecated and will be removed in a future version.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [15]:
#convert main functions to tensorflow functions to speed up training
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [16]:
#@tf.function
def train_agent(n_iterations):
    #reset metrics
    for m in training_metrics:
        m.reset()
    time_step = None
    policy_state = ()#agent.collect_policy.get_initial_state(tf_env.batch_size)
    agent.train_step_counter.assign(0)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                #plot metrics
                for train_metric in training_metrics:
                    train_metric.tf_summaries(train_step=tf.cast(agent.train_step_counter,tf.int64), step_metrics=training_metrics[:])
                train_summary_writer.flush()
                
        if iteration % 500 == 0:
            with train_summary_writer.as_default():
                #plot train loss          
                tf.summary.scalar('train_loss', train_loss.loss.numpy(), step=tf.cast(agent.train_step_counter,tf.int64))
                #plot NN weights
                for layer in  q_net.layers:
                    for weight in layer.weights:
                        tf.summary.histogram(weight.name,weight,step=tf.cast(agent.train_step_counter,tf.int64))
                train_summary_writer.flush()

In [17]:
train_dir = os.path.join(data_log_path,"logs/",run_label)   
train_summary_writer = tf.summary.create_file_writer(
            train_dir, flush_millis=10000)
#train_summary_writer.set_as_default()

In [18]:
# Launch TensorBoard with objects in the log directory
# This should launch tensorboard in your browser, but you may not see your metadata.
#%tensorboard --logdir=logs --reload_interval=15

In [19]:
#tf.summary.scalar('avgreturn', training_metrics[3].result().numpy(), step=tf.cast(agent.train_step_counter,tf.int64))

In [20]:
%time train_agent(pm['misc']['n_iterations'])

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


WARNING - 2021-04-12 18:39:05,205 - deprecation - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201: calling foldr_v2 (from tensorflow.python.ops.functional_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


46637 loss:2.341313780.44540456710109944
46965 loss:371.8857720.44310722602264485
0.5705791817550181
46966 loss:52.358230.43252033000807777
48012 loss:1.999088070.47276661477744786
48596 loss:375.0189590.4567765159426791
49999 loss:425.97836CPU times: user 3h 21min 41s, sys: 17min 14s, total: 3h 38min 55s
Wall time: 3h 3min 5s


In [21]:
#save model, parameters and code state
policy_dir = os.path.join(data_log_path,"policies", run_label)
tf_policy_saver = policy_saver.PolicySaver(agent.policy)
tf_policy_saver.save(policy_dir)

with open(os.path.join(data_log_path,"parameters", run_label+'.json'), 'w') as json_file:
  json.dump(pm, json_file)

src = os.path.join(current_path,"dqn_tf_agents.ipynb")
dst = os.path.join(data_log_path,"train_code", run_label+'.ipynb')
shutil.copyfile(src, dst)

src = os.path.join(current_path,"scan_gym/scan_gym/envs/ScannerEnv/scanner_env.py")
dst = os.path.join(data_log_path,"environment_code", run_label+'.py')
shutil.copyfile(src, dst)


WARNING - 2021-04-12 21:41:55,786 - save - Found untraced functions such as QNetwork_layer_call_fn, QNetwork_layer_call_and_return_conditional_losses, EncodingNetwork_layer_call_fn, EncodingNetwork_layer_call_and_return_conditional_losses, dense_1_layer_call_fn while saving (showing 5 of 30). These functions will not be directly callable after loading.
WARNING - 2021-04-12 21:41:55,817 - save - Found untraced functions such as QNetwork_layer_call_fn, QNetwork_layer_call_and_return_conditional_losses, EncodingNetwork_layer_call_fn, EncodingNetwork_layer_call_and_return_conditional_losses, dense_1_layer_call_fn while saving (showing 5 of 30). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/policies/20210412-183753/assets


INFO - 2021-04-12 21:41:55,950 - builder_impl - Assets written to: /home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/policies/20210412-183753/assets


'/home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/environment_code/20210412-183753.py'

In [22]:
replay_buffer.num_frames()

<tf.Tensor: shape=(), dtype=int64, numpy=200000>

In [30]:
dp = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/000'
env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp})
test_env = tf_py_environment.TFPyEnvironment( env )

In [31]:
state = test_env.reset()
#print(state.observation[0].numpy())
last_position = env.current_position
cont = 0
time_steps = 3000
#obs = deque(maxlen=time_steps)

for i in range(time_steps):
    #obs.append(state.observation[0].numpy())
    cont +=1
    action_step = agent.policy.action(state)#agent.policy.action(state)
    next_state = test_env.step(action_step.action)
    print(cont,':',last_position,state.observation[1].numpy(),env.action2move(action_step.action.numpy(),state.observation[1].numpy()[0][-1]) ) #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    state = copy.deepcopy(next_state)
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_images)
print(sorted(env.kept_images))
print(env.total_reward)
print(len(env.kept_images))
print(env.total_moves)
print(env.chamfer_from_collected())

1 : 502 [[0. 1. 0. 0. 1.]] 81111
2 : 502 [[0. 1. 1. 0. 1.]] 81111
3 : 502 [[0. 1. 2. 0. 1.]] 81111
4 : 502 [[0. 1. 3. 0. 1.]] 81111
5 : 502 [[0. 1. 4. 0. 1.]] 88888
6 : 502 [[0. 1. 0. 0. 1.]] 81111
7 : 502 [[0. 1. 1. 0. 1.]] 81111
8 : 502 [[0. 1. 2. 0. 1.]] 81111
9 : 502 [[0. 1. 3. 0. 1.]] 81111
10 : 502 [[0. 1. 4. 0. 1.]] 88888
11 : 502 [[0. 1. 0. 0. 1.]] 81111
12 : 502 [[0. 1. 1. 0. 1.]] 81111
13 : 502 [[0. 1. 2. 0. 1.]] 81111
14 : 502 [[0. 1. 3. 0. 1.]] 81111
15 : 502 [[0. 1. 4. 0. 1.]] 88888
16 : 502 [[0. 1. 0. 0. 1.]] 81111
17 : 502 [[0. 1. 1. 0. 1.]] 81111
18 : 502 [[0. 1. 2. 0. 1.]] 81111
19 : 502 [[0. 1. 3. 0. 1.]] 81111
20 : 502 [[0. 1. 4. 0. 1.]] 88888
21 : 502 [[0. 1. 0. 0. 1.]] 81111
22 : 502 [[0. 1. 1. 0. 1.]] 81111
23 : 502 [[0. 1. 2. 0. 1.]] 81111
24 : 502 [[0. 1. 3. 0. 1.]] 81111
25 : 502 [[0. 1. 4. 0. 1.]] 88888
26 : 502 [[0. 1. 0. 0. 1.]] 81111
27 : 502 [[0. 1. 1. 0. 1.]] 81111
28 : 502 [[0. 1. 2. 0. 1.]] 81111
29 : 502 [[0. 1. 3. 0. 1.]] 81111
30 : 502 [[0. 1. 4. 0. 

281 : 502 [[0. 1. 0. 0. 1.]] 81111
282 : 502 [[0. 1. 1. 0. 1.]] 81111
283 : 502 [[0. 1. 2. 0. 1.]] 81111
284 : 502 [[0. 1. 3. 0. 1.]] 81111
285 : 502 [[0. 1. 4. 0. 1.]] 88888
286 : 502 [[0. 1. 0. 0. 1.]] 81111
287 : 502 [[0. 1. 1. 0. 1.]] 81111
288 : 502 [[0. 1. 2. 0. 1.]] 81111
289 : 502 [[0. 1. 3. 0. 1.]] 81111
290 : 502 [[0. 1. 4. 0. 1.]] 88888
291 : 502 [[0. 1. 0. 0. 1.]] 81111
292 : 502 [[0. 1. 1. 0. 1.]] 81111
293 : 502 [[0. 1. 2. 0. 1.]] 81111
294 : 502 [[0. 1. 3. 0. 1.]] 81111
295 : 502 [[0. 1. 4. 0. 1.]] 88888
296 : 502 [[0. 1. 0. 0. 1.]] 81111
297 : 502 [[0. 1. 1. 0. 1.]] 81111
298 : 502 [[0. 1. 2. 0. 1.]] 81111
299 : 502 [[0. 1. 3. 0. 1.]] 81111
300 : 502 [[0. 1. 4. 0. 1.]] 88888
301 : 502 [[0. 1. 0. 0. 1.]] 81111
302 : 502 [[0. 1. 1. 0. 1.]] 81111
303 : 502 [[0. 1. 2. 0. 1.]] 81111
304 : 502 [[0. 1. 3. 0. 1.]] 81111
305 : 502 [[0. 1. 4. 0. 1.]] 88888
306 : 502 [[0. 1. 0. 0. 1.]] 81111
307 : 502 [[0. 1. 1. 0. 1.]] 81111
308 : 502 [[0. 1. 2. 0. 1.]] 81111
309 : 502 [[0. 1. 3.

521 : 502 [[0. 1. 0. 0. 1.]] 81111
522 : 502 [[0. 1. 1. 0. 1.]] 81111
523 : 502 [[0. 1. 2. 0. 1.]] 81111
524 : 502 [[0. 1. 3. 0. 1.]] 81111
525 : 502 [[0. 1. 4. 0. 1.]] 88888
526 : 502 [[0. 1. 0. 0. 1.]] 81111
527 : 502 [[0. 1. 1. 0. 1.]] 81111
528 : 502 [[0. 1. 2. 0. 1.]] 81111
529 : 502 [[0. 1. 3. 0. 1.]] 81111
530 : 502 [[0. 1. 4. 0. 1.]] 88888
531 : 502 [[0. 1. 0. 0. 1.]] 81111
532 : 502 [[0. 1. 1. 0. 1.]] 81111
533 : 502 [[0. 1. 2. 0. 1.]] 81111
534 : 502 [[0. 1. 3. 0. 1.]] 81111
535 : 502 [[0. 1. 4. 0. 1.]] 88888
536 : 502 [[0. 1. 0. 0. 1.]] 81111
537 : 502 [[0. 1. 1. 0. 1.]] 81111
538 : 502 [[0. 1. 2. 0. 1.]] 81111
539 : 502 [[0. 1. 3. 0. 1.]] 81111
540 : 502 [[0. 1. 4. 0. 1.]] 88888
541 : 502 [[0. 1. 0. 0. 1.]] 81111
542 : 502 [[0. 1. 1. 0. 1.]] 81111
543 : 502 [[0. 1. 2. 0. 1.]] 81111
544 : 502 [[0. 1. 3. 0. 1.]] 81111
545 : 502 [[0. 1. 4. 0. 1.]] 88888
546 : 502 [[0. 1. 0. 0. 1.]] 81111
547 : 502 [[0. 1. 1. 0. 1.]] 81111
548 : 502 [[0. 1. 2. 0. 1.]] 81111
549 : 502 [[0. 1. 3.

761 : 502 [[0. 1. 0. 0. 1.]] 81111
762 : 502 [[0. 1. 1. 0. 1.]] 81111
763 : 502 [[0. 1. 2. 0. 1.]] 81111
764 : 502 [[0. 1. 3. 0. 1.]] 81111
765 : 502 [[0. 1. 4. 0. 1.]] 88888
766 : 502 [[0. 1. 0. 0. 1.]] 81111
767 : 502 [[0. 1. 1. 0. 1.]] 81111
768 : 502 [[0. 1. 2. 0. 1.]] 81111
769 : 502 [[0. 1. 3. 0. 1.]] 81111
770 : 502 [[0. 1. 4. 0. 1.]] 88888
771 : 502 [[0. 1. 0. 0. 1.]] 81111
772 : 502 [[0. 1. 1. 0. 1.]] 81111
773 : 502 [[0. 1. 2. 0. 1.]] 81111
774 : 502 [[0. 1. 3. 0. 1.]] 81111
775 : 502 [[0. 1. 4. 0. 1.]] 88888
776 : 502 [[0. 1. 0. 0. 1.]] 81111
777 : 502 [[0. 1. 1. 0. 1.]] 81111
778 : 502 [[0. 1. 2. 0. 1.]] 81111
779 : 502 [[0. 1. 3. 0. 1.]] 81111
780 : 502 [[0. 1. 4. 0. 1.]] 88888
781 : 502 [[0. 1. 0. 0. 1.]] 81111
782 : 502 [[0. 1. 1. 0. 1.]] 81111
783 : 502 [[0. 1. 2. 0. 1.]] 81111
784 : 502 [[0. 1. 3. 0. 1.]] 81111
785 : 502 [[0. 1. 4. 0. 1.]] 88888
786 : 502 [[0. 1. 0. 0. 1.]] 81111
787 : 502 [[0. 1. 1. 0. 1.]] 81111
788 : 502 [[0. 1. 2. 0. 1.]] 81111
789 : 502 [[0. 1. 3.

1001 : 502 [[0. 1. 0. 0. 1.]] 81111
1002 : 502 [[0. 1. 1. 0. 1.]] 81111
1003 : 502 [[0. 1. 2. 0. 1.]] 81111
1004 : 502 [[0. 1. 3. 0. 1.]] 81111
1005 : 502 [[0. 1. 4. 0. 1.]] 88888
1006 : 502 [[0. 1. 0. 0. 1.]] 81111
1007 : 502 [[0. 1. 1. 0. 1.]] 81111
1008 : 502 [[0. 1. 2. 0. 1.]] 81111
1009 : 502 [[0. 1. 3. 0. 1.]] 81111
1010 : 502 [[0. 1. 4. 0. 1.]] 88888
1011 : 502 [[0. 1. 0. 0. 1.]] 81111
1012 : 502 [[0. 1. 1. 0. 1.]] 81111
1013 : 502 [[0. 1. 2. 0. 1.]] 81111
1014 : 502 [[0. 1. 3. 0. 1.]] 81111
1015 : 502 [[0. 1. 4. 0. 1.]] 88888
1016 : 502 [[0. 1. 0. 0. 1.]] 81111
1017 : 502 [[0. 1. 1. 0. 1.]] 81111
1018 : 502 [[0. 1. 2. 0. 1.]] 81111
1019 : 502 [[0. 1. 3. 0. 1.]] 81111
1020 : 502 [[0. 1. 4. 0. 1.]] 88888
1021 : 502 [[0. 1. 0. 0. 1.]] 81111
1022 : 502 [[0. 1. 1. 0. 1.]] 81111
1023 : 502 [[0. 1. 2. 0. 1.]] 81111
1024 : 502 [[0. 1. 3. 0. 1.]] 81111
1025 : 502 [[0. 1. 4. 0. 1.]] 88888
1026 : 502 [[0. 1. 0. 0. 1.]] 81111
1027 : 502 [[0. 1. 1. 0. 1.]] 81111
1028 : 502 [[0. 1. 2. 0. 1.]

1241 : 502 [[0. 1. 0. 0. 1.]] 81111
1242 : 502 [[0. 1. 1. 0. 1.]] 81111
1243 : 502 [[0. 1. 2. 0. 1.]] 81111
1244 : 502 [[0. 1. 3. 0. 1.]] 81111
1245 : 502 [[0. 1. 4. 0. 1.]] 88888
1246 : 502 [[0. 1. 0. 0. 1.]] 81111
1247 : 502 [[0. 1. 1. 0. 1.]] 81111
1248 : 502 [[0. 1. 2. 0. 1.]] 81111
1249 : 502 [[0. 1. 3. 0. 1.]] 81111
1250 : 502 [[0. 1. 4. 0. 1.]] 88888
1251 : 502 [[0. 1. 0. 0. 1.]] 81111
1252 : 502 [[0. 1. 1. 0. 1.]] 81111
1253 : 502 [[0. 1. 2. 0. 1.]] 81111
1254 : 502 [[0. 1. 3. 0. 1.]] 81111
1255 : 502 [[0. 1. 4. 0. 1.]] 88888
1256 : 502 [[0. 1. 0. 0. 1.]] 81111
1257 : 502 [[0. 1. 1. 0. 1.]] 81111
1258 : 502 [[0. 1. 2. 0. 1.]] 81111
1259 : 502 [[0. 1. 3. 0. 1.]] 81111
1260 : 502 [[0. 1. 4. 0. 1.]] 88888
1261 : 502 [[0. 1. 0. 0. 1.]] 81111
1262 : 502 [[0. 1. 1. 0. 1.]] 81111
1263 : 502 [[0. 1. 2. 0. 1.]] 81111
1264 : 502 [[0. 1. 3. 0. 1.]] 81111
1265 : 502 [[0. 1. 4. 0. 1.]] 88888
1266 : 502 [[0. 1. 0. 0. 1.]] 81111
1267 : 502 [[0. 1. 1. 0. 1.]] 81111
1268 : 502 [[0. 1. 2. 0. 1.]

1481 : 502 [[0. 1. 0. 0. 1.]] 81111
1482 : 502 [[0. 1. 1. 0. 1.]] 81111
1483 : 502 [[0. 1. 2. 0. 1.]] 81111
1484 : 502 [[0. 1. 3. 0. 1.]] 81111
1485 : 502 [[0. 1. 4. 0. 1.]] 88888
1486 : 502 [[0. 1. 0. 0. 1.]] 81111
1487 : 502 [[0. 1. 1. 0. 1.]] 81111
1488 : 502 [[0. 1. 2. 0. 1.]] 81111
1489 : 502 [[0. 1. 3. 0. 1.]] 81111
1490 : 502 [[0. 1. 4. 0. 1.]] 88888
1491 : 502 [[0. 1. 0. 0. 1.]] 81111
1492 : 502 [[0. 1. 1. 0. 1.]] 81111
1493 : 502 [[0. 1. 2. 0. 1.]] 81111
1494 : 502 [[0. 1. 3. 0. 1.]] 81111
1495 : 502 [[0. 1. 4. 0. 1.]] 88888
1496 : 502 [[0. 1. 0. 0. 1.]] 81111
1497 : 502 [[0. 1. 1. 0. 1.]] 81111
1498 : 502 [[0. 1. 2. 0. 1.]] 81111
1499 : 502 [[0. 1. 3. 0. 1.]] 81111
1500 : 502 [[0. 1. 4. 0. 1.]] 88888
1501 : 502 [[0. 1. 0. 0. 1.]] 81111
1502 : 502 [[0. 1. 1. 0. 1.]] 81111
1503 : 502 [[0. 1. 2. 0. 1.]] 81111
1504 : 502 [[0. 1. 3. 0. 1.]] 81111
1505 : 502 [[0. 1. 4. 0. 1.]] 88888
1506 : 502 [[0. 1. 0. 0. 1.]] 81111
1507 : 502 [[0. 1. 1. 0. 1.]] 81111
1508 : 502 [[0. 1. 2. 0. 1.]

1721 : 502 [[0. 1. 0. 0. 1.]] 81111
1722 : 502 [[0. 1. 1. 0. 1.]] 81111
1723 : 502 [[0. 1. 2. 0. 1.]] 81111
1724 : 502 [[0. 1. 3. 0. 1.]] 81111
1725 : 502 [[0. 1. 4. 0. 1.]] 88888
1726 : 502 [[0. 1. 0. 0. 1.]] 81111
1727 : 502 [[0. 1. 1. 0. 1.]] 81111
1728 : 502 [[0. 1. 2. 0. 1.]] 81111
1729 : 502 [[0. 1. 3. 0. 1.]] 81111
1730 : 502 [[0. 1. 4. 0. 1.]] 88888
1731 : 502 [[0. 1. 0. 0. 1.]] 81111
1732 : 502 [[0. 1. 1. 0. 1.]] 81111
1733 : 502 [[0. 1. 2. 0. 1.]] 81111
1734 : 502 [[0. 1. 3. 0. 1.]] 81111
1735 : 502 [[0. 1. 4. 0. 1.]] 88888
1736 : 502 [[0. 1. 0. 0. 1.]] 81111
1737 : 502 [[0. 1. 1. 0. 1.]] 81111
1738 : 502 [[0. 1. 2. 0. 1.]] 81111
1739 : 502 [[0. 1. 3. 0. 1.]] 81111
1740 : 502 [[0. 1. 4. 0. 1.]] 88888
1741 : 502 [[0. 1. 0. 0. 1.]] 81111
1742 : 502 [[0. 1. 1. 0. 1.]] 81111
1743 : 502 [[0. 1. 2. 0. 1.]] 81111
1744 : 502 [[0. 1. 3. 0. 1.]] 81111
1745 : 502 [[0. 1. 4. 0. 1.]] 88888
1746 : 502 [[0. 1. 0. 0. 1.]] 81111
1747 : 502 [[0. 1. 1. 0. 1.]] 81111
1748 : 502 [[0. 1. 2. 0. 1.]

1961 : 502 [[0. 1. 0. 0. 1.]] 81111
1962 : 502 [[0. 1. 1. 0. 1.]] 81111
1963 : 502 [[0. 1. 2. 0. 1.]] 81111
1964 : 502 [[0. 1. 3. 0. 1.]] 81111
1965 : 502 [[0. 1. 4. 0. 1.]] 88888
1966 : 502 [[0. 1. 0. 0. 1.]] 81111
1967 : 502 [[0. 1. 1. 0. 1.]] 81111
1968 : 502 [[0. 1. 2. 0. 1.]] 81111
1969 : 502 [[0. 1. 3. 0. 1.]] 81111
1970 : 502 [[0. 1. 4. 0. 1.]] 88888
1971 : 502 [[0. 1. 0. 0. 1.]] 81111
1972 : 502 [[0. 1. 1. 0. 1.]] 81111
1973 : 502 [[0. 1. 2. 0. 1.]] 81111
1974 : 502 [[0. 1. 3. 0. 1.]] 81111
1975 : 502 [[0. 1. 4. 0. 1.]] 88888
1976 : 502 [[0. 1. 0. 0. 1.]] 81111
1977 : 502 [[0. 1. 1. 0. 1.]] 81111
1978 : 502 [[0. 1. 2. 0. 1.]] 81111
1979 : 502 [[0. 1. 3. 0. 1.]] 81111
1980 : 502 [[0. 1. 4. 0. 1.]] 88888
1981 : 502 [[0. 1. 0. 0. 1.]] 81111
1982 : 502 [[0. 1. 1. 0. 1.]] 81111
1983 : 502 [[0. 1. 2. 0. 1.]] 81111
1984 : 502 [[0. 1. 3. 0. 1.]] 81111
1985 : 502 [[0. 1. 4. 0. 1.]] 88888
1986 : 502 [[0. 1. 0. 0. 1.]] 81111
1987 : 502 [[0. 1. 1. 0. 1.]] 81111
1988 : 502 [[0. 1. 2. 0. 1.]

In [25]:
#batched_py_environment.BatchedPyEnvironment(envs=[suite.load(...) for _ in range(n)])

In [26]:
#q_net.layers[0].weights[0].shape